In [ ]:
# import preprocess as prep
import SLR_model
import SLR_model_GRU
import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
from tensorflow import keras
import time
import os
import json
import importlib
from matplotlib import pyplot as plt
import preprocess
# from preprocess import person



# model will output multiple(5) results/sec, how are we gonna handle it?


In [ ]:
load_size = 3000 # number of data to be loaded at once
epochs = 50
run_time=2
batch_size = 16
save_dir = "saves_GRU"
load_dir = "saves_GRU"
model = SLR_model_GRU.get_model()

In [ ]:
 # reload model file
end_file=preprocess.getoutputdir()

save_suffix = time.strftime("%m-%d-%H", time.localtime(time.time()))
ckpt_name=save_suffix+"-"+str(epochs)+"epochs-"+str(run_time)+"times"

check_path = os.path.join(save_dir,'ckpt',ckpt_name)
hist_path = os.path.join(save_dir, "hist",ckpt_name+".json")

model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
filepath=f'{check_path}.keras',
monitor='categorical_accuracy',
mode='max',
save_freq='epoch',
save_best_only=True)


start_person=1
start_count=1
start_word=" "
with open(os.path.join('logs',ckpt_name+'.txt'), 'a') as logs:
    for i in range(1,17):
        l_raws=[]
        r_raws=[]
        p_raws=[]
        y_raws=[]
        loss_weights_raws=[]
        if start_person>i:
            continue
        for k in range(1,run_time+1):
            if start_count>k:
                continue
            elif start_count==k:
                start_count=0
            for j in sorted(os.listdir(os.path.join(preprocess.getoutputdir(),str(i)))):
                if len(l_raws)==0:
                    start_word = j
                else:
                    end_word = j
                l_raw, r_raw, p_raw, y_raw, loss_weights_raw = preprocess.load_data(f"{i}/{j}")
                l_raws.append(l_raw)
                r_raws.append(r_raw)
                p_raws.append(p_raw)
                y_raws.append(y_raw)
                loss_weights_raws.append(loss_weights_raw)
                # print(p_raw.shape)
                # break
                if len(l_raws)>=load_size:
                    
                    logs.write(f'{ time.strftime("%H-%M-%S", time.localtime(time.time()))}:{k}) person:{i} : {start_word} ~ {end_word}\n')  # 한 줄 쓰기
                    l_train, each = SLR_model_GRU.serialize(l_raws)
                    r_train, each = SLR_model_GRU.serialize(r_raws)
                    p_train, each, sample_weights = SLR_model_GRU.serialize(p_raws, stride=2, loss_weights_list=loss_weights_raws)
                    x_train = (l_train, r_train, p_train)
                    
                    y_train = np.repeat(y_raws, each)
                    y_train = SLR_model_GRU.encode_onehot2d(y_train)
                    
                    dataset = SLR_model_GRU.convert_to_dataset(x_train, y_train, batch_size, sample_weights)
                    hist = model.fit(dataset, epochs=epochs, callbacks=[model_checkpoint_callback])
                    
                    # model.save(os.path.join(save_dir, 'model', f"{ckpt_name}.h5"))

                    # converter = tf.lite.TFLiteConverter.from_keras_model(model)
                    # tflite_model = converter.convert()
                    # with open(os.path.join(save_dir, 'model', f"{ckpt_name}.tflite"), 'wb') as tflite_file:
                    #     tflite_file.write(tflite_model)

                    with open(hist_path, 'w') as file:
                        json.dump(hist.history, file)
                    l_raws.clear()
                    r_raws.clear()
                    p_raws.clear()
                    y_raws.clear()
                    loss_weights_raws.clear()
            if len(l_raws)>0:
                l_train, each = SLR_model_GRU.serialize(l_raws)
                r_train, each = SLR_model_GRU.serialize(r_raws)
                p_train, each, sample_weights = SLR_model_GRU.serialize(p_raws, stride=2, loss_weights_list=loss_weights_raws)
                x_train = (l_train, r_train, p_train)
                
                y_train = np.repeat(y_raws, each)
                y_train = SLR_model_GRU.encode_onehot2d(y_train)
                
                
                dataset = SLR_model_GRU.convert_to_dataset(x_train, y_train, batch_size, sample_weights)
                hist = model.fit(dataset, epochs=epochs, callbacks=[model_checkpoint_callback])
                
                # model.save(os.path.join(save_dir, 'model', f"{ckpt_name}.h5"))

                # converter = tf.lite.TFLiteConverter.from_keras_model(model)
                # tflite_model = converter.convert()
                # with open(os.path.join(save_dir, 'model', f"{ckpt_name}.tflite"), 'wb') as tflite_file:
                #     tflite_file.write(tflite_model)

                with open(hist_path, 'w') as file:
                    json.dump(hist.history, file)



In [ ]:
# Configurations
load_size = 3000 # number of data to be loaded at once
epochs = 50
batch_size = 16
save_dir = "saves"
load_dir = "saves"
# load_path = "C:/Users/jerry/Desktop/hly/2024-2/cap/BN_SLR/BomNae-SLR/checkpoints.keras"


In [ ]:
model = SLR_model.get_model()

In [ ]:
 # reload model file
importlib.reload(SLR_model)
importlib.reload(preprocess)
end_file=preprocess.getoutputdir()

save_suffix = time.strftime("_%d-%H-%M", time.localtime(time.time()))


# checkTime="_14-16-16"
# check_path = os.path.join(save_dir, "check"+checkTime)
# hist_path = os.path.join(save_dir, "hist"+checkTime+".json")

check_path = os.path.join(save_dir, "check"+save_suffix+"_50epochs_3time")
hist_path = os.path.join(save_dir, "hist"+save_suffix+".json")

model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
filepath=f'{check_path}.keras',
monitor='categorical_accuracy',
mode='max',
save_freq='epoch',
save_best_only=True)


start_person=1
start_count=1
start_word=" "
for i in range(1,17):
    l_raws=[]
    r_raws=[]
    p_raws=[]
    y_raws=[]
    loss_weights_raws=[]
    if start_person>i:
        continue
    for k in range(1,2):
        if start_count>k:
            continue
        elif start_count==k:
            start_count=0
        for j in sorted(os.listdir(os.path.join(preprocess.getoutputdir(),str(i)))):
            if len(l_raws)==0:
                start_word = j
            else:
                end_word = j
            l_raw, r_raw, p_raw, y_raw, loss_weights_raw = preprocess.load_data(f"{i}/{j}")
            l_raws.append(l_raw)
            r_raws.append(r_raw)
            p_raws.append(p_raw)
            y_raws.append(y_raw)
            loss_weights_raws.append(loss_weights_raw)
    
            if len(l_raws)>=load_size:
                with open(str(i)+'training_log.txt', 'a') as logs:
                    logs.write(f'{ time.strftime("%H-%M-%S", time.localtime(time.time()))}:{k}) person:{i} : {start_word} ~ {end_word}\n')  # 한 줄 쓰기
                print(f"{k}) person:{i} : {start_word} ~ {end_word}\n\n\n\n")
                l_train, each = SLR_model.serialize(l_raws)
                r_train, each = SLR_model.serialize(r_raws)
                p_train, each, sample_weights = SLR_model.serialize(p_raws, stride=2, loss_weights_list=loss_weights_raws)
                x_train = (l_train, r_train, p_train)
                
                y_train = np.repeat(y_raws, each)
                y_train = SLR_model.encode_onehot2d(y_train)
    
                dataset = SLR_model.convert_to_dataset(x_train, y_train, batch_size, sample_weights)
                hist = model.fit(dataset, epochs=epochs, callbacks=[model_checkpoint_callback])
                
                with open(hist_path, 'w') as file:
                    json.dump(hist.history, file)
                l_raws.clear()
                r_raws.clear()
                p_raws.clear()
                y_raws.clear()
                loss_weights_raws.clear()
        if len(l_raws)>0:
            l_train, each = SLR_model.serialize(l_raws)
            r_train, each = SLR_model.serialize(r_raws)
            p_train, each, sample_weights = SLR_model.serialize(p_raws, stride=2, loss_weights_list=loss_weights_raws)
            x_train = (l_train, r_train, p_train)
            
            y_train = np.repeat(y_raws, each)
            y_train = SLR_model.encode_onehot2d(y_train)
    
            dataset = SLR_model.convert_to_dataset(x_train, y_train, batch_size, sample_weights)
            hist = model.fit(dataset, epochs=epochs, callbacks=[model_checkpoint_callback])
            
            with open(hist_path, 'w') as file:
                json.dump(hist.history, file)

    

In [ ]:
# hand input shape: (batch, time, h, w, channels)
# pose input shape: (batch, time, channel, features)

In [ ]:
# # 세이브된 모델 로드
# load_path = os.join(load_dir, "check_00-00-00.keras")
# model = SLR_model.load_model(load_path)

In [ ]:
# # 안될때 (강제중지 + 초기화)
# tf.keras.backend.clear_session()

In [ ]:
# l_raws=[]
# r_raws=[]
# p_raws=[]
# y_raws=[]


# save_suffix = time.strftime("_%d-%H-%M", time.localtime(time.time()))
# check_path = os.path.join(save_dir, "check"+save_suffix)
# hist_path = os.path.join(save_dir, "hist"+save_suffix+".json")

# model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
#     filepath=f'{check_path}.keras',
#     monitor='binary_accuracy',
#     mode='max',
#     save_freq='epoch',
#     save_best_only=True)



# l_raw, r_raw, p_raw, y_raw = preprocess.load_data(f"1/1")

# l_raws.append(l_raw)
# r_raws.append(r_raw)
# p_raws.append(p_raw)
# y_raws.append(y_raw)


# l_train, each = SLR_model.serialize(l_raws)
# r_train, each = SLR_model.serialize(r_raws)
# p_train, each = SLR_model.serialize(p_raws, stride=2)
# y_train = np.repeat(y_raws, each)
# y_train = SLR_model.num_arr2bin(y_train, 12)
# x_train = (l_train, r_train, p_train)

# # dataset = SLR_model.convert_to_dataset(x_train, y_train, batch_size)
# hist = model.fit(x_train, y_train, batch_size = 2, epochs=epochs, callbacks=[model_checkpoint_callback])

# with open(hist_path, 'w') as file:
#     json.dump(hist.history, file)


In [ ]:
plt.plot(range(len(hist.history['loss'])), hist.history['loss'])
plt.scatter(range(len(hist.history['loss'])), hist.history['loss'])
plt.xlabel("epochs")
plt.ylabel("loss")
plt.legend(["loss"])
plt.ylim((0,1))
plt.show()

In [ ]:
print(os.listdir(os.path.join(preprocess.getoutputdir(),str(15))))

In [ ]:
model.metrics_names